<font color="#CA3532"><h1 align="left">Transformación de datos</h1></font>

**Manuel Sánchez-Montañés**

## <font color="#CA3532">Transformación de los datos originales</font>

Después de la limpieza de datos, es posible que tengamos que realizar algunas transformaciones en los datos. Veremos en detalle cómo realizar las siguientes tareas:

- Normalización y / o estandarización de datos.
- Agregación de datos.
- Discretización de datos y / o dicotomización.
- Manipulación de fechas.

Otras transformaciones posibles que no están cubiertas en estas notas incluyen suavizado, segmentación de datos, compresión de series temporales, etc.

Primero hacemos todos los imports necesarios:

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

matplotlib.style.use('ggplot')

### <font color="#CA3532">Normalización de los datos</font>

La normalización de datos es muy importante, especialmente cuando se han medido diferentes atributos con diferentes escalas. Aplicamos la normalización solo a los atributos numéricos. Una forma de normalizar bastante común es la estandarización, que consiste en restar la media y escalar para tener una media igual a 0 y una desviación estándar igual a 1.

In [ ]:
# Leer dataset "labor" y mantener sólo los atributos numéricos:
data = pd.read_csv("../datasets/labor.csv", na_values = ["?"], sep = ",")
data = data.loc[:, data.dtypes != object]
data[:10]

In [ ]:
data.mean()

In [ ]:
data.std()

In [ ]:
# Estandarización:
data = (data - data.mean()) / data.std() # También se conoce como el "Z-Score"
data[:10]

In [ ]:
# Chequear que las medias son ahora 0 (o prácticamente 0):
data.mean()

In [ ]:
# Chequear que las stds son ahora 1:
data.std()

### <font color="#CA3532">Agregación de datos</font>


In [ ]:
# Load adult data set:
data = pd.read_csv("../datasets/adult.csv", na_values = ["?"], sep = ",")
data[:10]

In [ ]:
# Agregar por campo education, calculando medias de atributos numéricos:
grouped = data.groupby(data["education"])
filtro  = data.columns[data.dtypes != object] # nombres de columnas cuyo tipo no es "object"
education_means = grouped[filtro].agg("mean")
education_means

In [ ]:
# Mostrar gráfica media hours-per-week versus education:
var = "hours-per-week"
h = education_means[var].sort_values().plot(kind = 'bar', figsize = (8, 5),
                                            title = var, color = 'firebrick')

In [ ]:
data.head(5)

In [ ]:
# Agregar por education, calculando frecuencia de salary >50K:
def rate50(x):
    return float(np.sum(x == ">50K")) / len(x)
    
grouped = data["class"].groupby(data["education"])
education_salary_50K = grouped.aggregate(rate50)
education_salary_50K

In [ ]:
# Gráfica de frecuencia de salary >50K versus education:
h = education_salary_50K.sort_values().plot(kind = 'bar', figsize = (8, 5),
                                            title = "rate >50K", color = 'firebrick')

### <font color="#CA3532">Discretización de los datos</font>

In [ ]:
# Cargar dataset "adult" y mantener los primeros 6 atributos:
data = pd.read_csv("../datasets/adult.csv", na_values = ["?"], sep = ",")
data = data.iloc[:,:6]
data[:10]

In [ ]:
# Discretizar "age" en 5 grupos:
age = data["age"]
bins = [0, 20, 30, 45, 65, 100]
age_discretized = pd.cut(age, bins)
age_discretized[:10]

In [ ]:
# Número de casos en cada rango de edad:
age_discretized.value_counts().sort_index()

In [ ]:
# Añadir edad discretizada a los datos: 
data.insert(1, "age-discretized", age_discretized)
data[:10]

In [ ]:
data.dtypes

### <font color="#CA3532">Dicotomización</font>

Convertir una variable categórica con *k* valores diferentes en *k* atributos binarios diferentes ("*one-hot* encoding").

In [ ]:
# cargar el dataset "adult" y mantener los 6 primeros atributos:
data = pd.read_csv("../datasets/adult.csv", na_values = ["?"], sep = ",")
data = data.iloc[:,:6]
data[:10]

In [ ]:
data["workclass"].value_counts()

In [ ]:
pd.options.display.max_columns = None

In [ ]:
pd.get_dummies(data).head(5)

In [ ]:
pd.get_dummies(data)

In [ ]:
# Dicotomizar la columna "marital-status":
dummies = pd.get_dummies(data["marital-status"])
dummies[:10]

In [ ]:
#?pd.merge

In [ ]:
# Combinar las dos matrices de datos:
new_data = pd.merge(data, dummies, left_index = True, right_index = True)
new_data[:10]

### <font color="#CA3532">Manipulación de fechas</font>

In [ ]:
# Simple example with dates:
data = pd.DataFrame({"id": [1, 2, 3, 4, 5],
                     "birth": ["3/27/1989", "12/26/1998", "4/19/1972",
                               "7/21/1967", "2/22/1976"]}) 
data

In [ ]:
data.dtypes

In [ ]:
# Convertir la cadena "birth" a un objeto "datetime" de Python: 
from datetime import datetime

In [ ]:
# Algunas pruebas:
ref = datetime.strptime("150318","%d%m%y")
#ref = datetime.strptime("15032018","%d%m%Y")
nueva = datetime.strptime("23-04-2018","%d-%m-%Y")

ref

In [ ]:
nueva

In [ ]:
#(nueva-ref).days()
a=(nueva-ref)
a.days

In [ ]:
data["birth"]

In [ ]:
def fun(x):
    return datetime.strptime(x, "%m/%d/%Y")

data["birth2"] = data["birth"].apply(fun)

In [ ]:
data

In [ ]:
# Convertir "birth" a edad (en años) usando la
# fecha 4/10/2010 como referencia:
#reference = datetime(2010, 4, 10, 0, 0, 0, 0)
reference = datetime.today()

def myfun(x):
    return (reference - x).days/365.25

data["age"] = data["birth2"].apply(myfun)
data

### <font color="#CA3532">Ejercicio</font>

Carga la base de datos *loan* y transforma las variables que lo necesiten.